In [ ]:
# default_exp utils.dict

In [ ]:
%load_ext autoreload
%autoreload 2

# Modify python dictionaries

> Various methods to modify and work with python dictionaries

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

%load_ext autoreload
%autoreload 2

In [ ]:
# export 
import pandas as pd
import numpy as np

## Modify values 

In [ ]:
# export
def apply_to_vals(nested_dict, func, use_key:bool=False):
    '''Applies a function "func" to all non-dict values of a (nested) dictionary'''
    new_dict = dict()

    for k, v in nested_dict.items():
        if isinstance(v, dict):
            new_dict[k] = apply_to_vals(v, func, use_key)
        else:
            if use_key:
                new_dict[k] = func(k, v)
            else:
                new_dict[k] = func(v)
                
    return new_dict

In [ ]:
my_dict = {"a": {"b": 1, "c": 2}, "c": {"d": 3, "e": 4}}
apply_to_vals(my_dict, np.square)

{'a': {'b': 1, 'c': 4}, 'c': {'d': 9, 'e': 16}}

In [ ]:
# hide

# apply_to_vals unit tests
# test empty 
my_dict = {}
new_dict = apply_to_vals(my_dict, np.square)
test_eq(new_dict, {})

# test non nested dict 
my_dict = {"a": 1, "b": 2, "c": 3, "d": 4}
new_dict = apply_to_vals(my_dict, np.square)
test_eq(new_dict, {"a": 1, "b": 4, "c": 9, "d": 16})

# test nested dict (both child dicts contain same number of keys)
my_dict = {"a": {"b": 1, "c": 2}, "d": {"e": 3, "f": 4}}
new_dict = apply_to_vals(my_dict, np.square)
test_eq(new_dict, {"a": {"b": 1, "c": 4}, "d": {"e": 9, "f": 16}})

# test nested dict (both child dicts contain uneven number of keys)
my_dict = {"a": {"b": 1, "c": 2}, "d": {"e": 3}}
new_dict = apply_to_vals(my_dict, np.square)
test_eq(new_dict,  {"a": {"b": 1, "c": 4}, "d": {"e": 9}})

# test nested dict (only one key has a child dict)
my_dict =  {"a": {"b": 1, "c": 2}, "d": 3}
new_dict = apply_to_vals(my_dict, np.square)
test_eq(new_dict,   {"a": {"b": 1, "c": 4}, "d": 9})

# test nested dict (child dict is empty)
my_dict =  {"a": {"b": {}}}
new_dict = apply_to_vals(my_dict, np.square)
test_eq(new_dict,   {"a": {"b": {}}})

## Modify keys

In [ ]:
# export
def remove_key(nested_dict, key, inplace:bool=True):
    '''Removes a key, together with its value, from a (nested) dict)'''
    
    new_dict = dict()
    
    for k, v in nested_dict.items():
        if isinstance(v, dict):
            if k!=key:
                new_dict[k] = remove_key(v, key) 
        else:
            if k!=key:
                new_dict[k] = v
                
    return new_dict

In [ ]:
# hide

# remove_key unit tests
# test empty 
my_dict = {}
new_dict = remove_key(my_dict, "a")
test_eq(new_dict, {})

# test non nested dict 
my_dict = {"a": 1, "b": 2, "c": 3, "d": 4}
new_dict = remove_key(my_dict, "a")
test_eq(new_dict, {"b": 2, "c": 3, "d": 4})

# test nested dict (one child dict contains key)
my_dict = {"a": {"b": 1, "c": 2}, "d": {"e": 3, "f": 4}}
new_dict = remove_key(my_dict, "c")
test_eq(new_dict, {"a": {"b": 1}, "d": {"e": 3, "f": 4}})

# test nested dict (both child dicts contains key)
my_dict = {"a": {"b": 1, "c": 2}, "d": {"e": 3, "c": 4}}
new_dict = remove_key(my_dict, "c")
test_eq(new_dict, {"a": {"b": 1}, "d": {"e": 3}})

## Modify structure

In [ ]:
# export
def flatten_intra(nested_dict, include_key:bool=False):
    new_dict = dict()
    intra_li = list()

    for k, v in nested_dict.items():
        if isinstance(v, dict):
            new_dict[k] = flatten_intra(v, include_key)
        else:
            if include_key:
                intra_li.append((k, v))
            else:
                intra_li.append(v)

    if intra_li:
        return intra_li
    else:
        return new_dict

In [ ]:
# export
def flatten_inter(nested_dict, include_key:bool=False):
    
    def flatten_inter_helper(nested_dict):
        new_dict = dict()
        intra_dict = dict()

        for k, v in nested_dict.items():
            if isinstance(v, dict):
                new_dict[k] = flatten_inter_helper(v)
            else:
                for kk, vv in v:
                    if intra_dict.get(kk) is None:
                        intra_dict[kk] = [(k, vv),] if include_key else [vv,]
                    else:
                        if include_key:
                            intra_dict[kk].append((k,vv))
                        else:
                            intra_dict[kk].append(vv)

        if intra_dict:
            return intra_dict
        else:
            return new_dict
        
    temp = flatten_intra(nested_dict, include_key=True)        
    return flatten_inter_helper(temp)
    